In [1]:
import pandas as pd
from Cleaner import BreadBasketCleaner
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


cleaner = BreadBasketCleaner()
steps = [
    cleaner,
    PCA(n_components=4)
]

pipe = make_pipeline(*steps)

data = pd.read_csv('../data/BreadBasket_DMS.csv')
X_feats = pipe.fit_transform(data)

pred = KMeans(n_clusters=5)
label = pred.fit_predict(X_feats)

print(pred.cluster_centers_)
predicted = pd.DataFrame(X_feats)
predicted["cluster"] = label
predicted

[[ 2.83490554 -1.21220065 -0.04398114  0.01247534]
 [-2.26482007  2.07942022  0.00872772  0.02010612]
 [ 0.24265063 -1.36914196  0.02006872 -0.01767048]
 [ 1.75651991  2.26959904  0.01757979 -0.01466356]
 [-3.04568601 -1.3853098  -0.01493306  0.01133337]]


,0,1,2,3,cluster
0,3.191589,-2.739883,-0.957769,0.191492,0
1,2.222059,-2.754363,-1.306679,1.015851,0
2,2.101788,-2.738647,0.393091,0.027655,0
3,2.193126,-2.737282,-0.612137,-0.735908,0
4,2.200950,-2.756539,0.004408,0.324062,0
...,...,...,...,...,...
9526,-1.797096,-2.730057,-1.130414,0.184986,4
9527,-1.900147,-2.770524,0.362417,-0.489125,4
9528,-1.843963,-2.756390,1.163047,-0.529865,4
9529,-1.818205,-2.732233,0.180674,-0.506804,4


In [2]:
df_orign = cleaner.fit_transform(data)
df_orign["labels"] = label
df_orign.head()

,OTHER_FOOD,beverage,bread,breakfast,breakfast_pastry,condiments,dessert,kids,lunch,snacks,hour,day,labels
Transaction,,,,,,,,,,,,,
1,0,0,1,0,0,0,0,0,0,0,9,6,0
2,0,0,2,0,0,0,0,0,0,0,10,6,0
3,0,1,0,0,0,1,1,0,0,0,10,6,0
4,0,0,0,0,1,0,0,0,0,0,10,6,0
5,0,1,1,0,1,0,0,0,0,0,10,6,0


In [3]:
df_orign.labels.value_counts()

2    2547
3    1948
0    1701
4    1676
1    1659
Name: labels, dtype: int64

In [4]:
pd.crosstab(df_orign.hour,df_orign.labels)

labels,0,1,2,3,4
hour,,,,,
1,1,0,0,0,0
7,9,0,0,7,0
8,232,0,0,143,0
9,635,0,0,372,0
10,824,0,0,443,0
11,0,0,962,483,0
12,0,0,847,500,0
13,0,425,738,0,0
14,0,553,0,0,577


In [5]:
pd.crosstab(df_orign.day,df_orign.labels)

labels,0,1,2,3,4
day,,,,,
0,0,485,0,650,0
1,0,531,0,672,0
2,3,490,0,626,2
3,353,153,502,0,244
4,469,0,586,0,433
5,602,0,814,0,652
6,274,0,645,0,345


In [6]:
x = BreadBasketCleaner(withDays=True,withItems=True).fit_transform(data)
x["label"] = df_orign.labels
x.head()

,Item,OTHER_FOOD,beverage,bread,breakfast,breakfast_pastry,condiments,dessert,kids,lunch,snacks,hour,day,label
Transaction,,,,,,,,,,,,,,
1,[Bread],0,0,1,0,0,0,0,0,0,0,9,Sunday,0
2,"[Scandinavian, Scandinavian]",0,0,2,0,0,0,0,0,0,0,10,Sunday,0
3,"[Hot chocolate, Jam, Cookies]",0,1,0,0,0,1,1,0,0,0,10,Sunday,0
4,[Muffin],0,0,0,0,1,0,0,0,0,0,10,Sunday,0
5,"[Coffee, Pastry, Bread]",0,1,1,0,1,0,0,0,0,0,10,Sunday,0


In [9]:
# only in pandas 0.25 
# pip3 install --upgrade pandas
exploded_labels = x.explode('Item')
exploded_labels.head()

,Item,OTHER_FOOD,beverage,bread,breakfast,breakfast_pastry,condiments,dessert,kids,lunch,snacks,hour,day,label
Transaction,,,,,,,,,,,,,,
1,Bread,0,0,1,0,0,0,0,0,0,0,9,Sunday,0
2,Scandinavian,0,0,2,0,0,0,0,0,0,0,10,Sunday,0
2,Scandinavian,0,0,2,0,0,0,0,0,0,0,10,Sunday,0
3,Hot chocolate,0,1,0,0,0,1,1,0,0,0,10,Sunday,0
3,Jam,0,1,0,0,0,1,1,0,0,0,10,Sunday,0


In [11]:
a = exploded_labels.groupby(['label']).Item.value_counts()
b = a.to_frame("counts").reset_index()
b.set_index("Item", inplace=True)
b.groupby('label').counts.nlargest(5)

label  Item         
0      Coffee            986
       Bread             735
       Pastry            283
       Medialuna         224
       Tea               169
1      Coffee            893
       Bread             491
       Tea               355
       Cake              257
       Sandwich          210
2      Coffee           1494
       Bread             909
       Tea               326
       NONE              295
       Sandwich          294
3      Coffee           1217
       Bread             672
       Pastry            267
       Tea               247
       NONE              151
4      Coffee            881
       Bread             518
       Tea               338
       Cake              302
       Hot chocolate     160
Name: counts, dtype: int64